# CAREFUL, ONE RUN CAN COST QUITE FAST A LOT, MAYBE CHANGE TO gpt-4o-mini

In [ ]:
model_name = "gpt-4o"

In [1]:
from dotenv import load_dotenv

# you need to create a .env file in the root of the project with following key:
# OPENAI_API_KEY=
load_dotenv()

True

In [2]:
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool(verbose=True)]

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI


from langgraph.prebuilt import create_react_agent

SYSTEM_PROMPT = """
You are a helpful assistant. You are executing a plan to answer a process mining task question.
For this you have two dataset available that the python tool can load:
- Data/DomesticDeclarations.xes
- Data/InternationalDeclarations.xes
You can get started with:

```python
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer

# Load the event logs
domestic_log = xes_importer.apply('Data/DomesticDeclarations.xes')
international_log = xes_importer.apply('Data/InternationalDeclarations.xes')

# Print the summary of the logs to understand the structure
domestic_summary = pm4py.get_event_attributes(domestic_log)
international_summary = pm4py.get_event_attributes(international_log)
```
"""

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="messages")
])
prompt.pretty_print()

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model=model_name)
agent_executor = create_react_agent(llm, tools, state_modifier=prompt)


================================ System Message ================================


You are a helpful assistant. You are executing a plan to answer a process mining task question.
For this you have two dataset available that the python tool can load:
- Data/DomesticDeclarations.xes
- Data/InternationalDeclarations.xes
You can get started with:

```python
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer

# Load the event logs
domestic_log = xes_importer.apply('Data/DomesticDeclarations.xes')
international_log = xes_importer.apply('Data/InternationalDeclarations.xes')

# Print the summary of the logs to understand the structure
domestic_summary = pm4py.get_event_attributes(domestic_log)
international_summary = pm4py.get_event_attributes(international_log)
```


============================= Messages Placeholder =============================

{messages}


In [4]:
# from IPython.display import Markdown, display
# test_runnable = agent_executor
# md_answer =test_runnable.invoke({"messages": [("user", "What attributes do the logs have?")]})["messages"][-1].content

# display(Markdown(md_answer))

In [5]:
import operator
from typing import Annotated, List, Tuple, TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [6]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Plan(BaseModel):
    """Plan to follow to solve the process mining task using python."""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )


In [7]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the process mining task, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
The Executer Agent will have a PythonREPL tool to execute your plan and data will be available.""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model=model_name, temperature=0
).with_structured_output(Plan)

In [8]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = replanner_prompt | ChatOpenAI(
    model=model_name, temperature=0
).with_structured_output(Act)

In [9]:
from typing import Literal


async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute) -> Literal["agent", "__end__"]:
    if "response" in state and state["response"]:
        return "__end__"
    else:
        return "agent"


In [10]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


In [11]:
# from IPython.display import Image, display

# display(Image(app.get_graph(xray=True).draw_mermaid_png()))



In [12]:
from IPython.display import display, Markdown
import json

config = {"recursion_limit": 10}
inputs = {"input": "Can you think of useful KPIs for the processes?"}

async for event in app.astream_events(inputs, config, version="v1"):
    if event["event"] == "on_chat_model_end":
        if event["data"]["output"]["generations"][0][0]["message"].content:
            display(Markdown(event["data"]["output"]["generations"][0][0]["message"].content))

/home/keno/.cache/pypoetry/virtualenvs/code-ha-process-mining-kRqu1UEl-py3.11/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


To define the key objectives of the process being analyzed, we need to understand the context and purpose of the processes captured in the event logs. Let's start by examining the summaries of the DomesticDeclarations.xes and InternationalDeclarations.xes logs to get insights into the activities, cases, and event attributes. This will help us understand the processes better and define their key objectives.

I'll proceed to load the logs and print their summaries.

Python REPL can execute arbitrary code. Use with caution.


parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

...

The summaries of both the Domestic and International event logs indicate that they contain the following attributes:
- `concept:name`: The name of the event.
- `org:resource`: The resource (person or system) involved in the event.
- `id`: The unique identifier for each event.
- `time:timestamp`: The timestamp of the event.
- `org:role`: The role associated with the resource.

Given this structure, we can measure the KPIs as follows:

1. **Cycle Time**:
   - **Measurable**: Calculate the difference between the timestamps of the first and last events for each case (declaration).

2. **Conformance Rate**:
   - **Measurable**: Use process mining techniques (e.g., conformance checking) to compare the actual event log to a reference model.

3. **Throughput Rate**:
   - **Measurable**: Count the number of declarations processed within a specific time period.

4. **Rework Rate**:
   - **Measurable**: Identify events that indicate rework (e.g., repeated tasks) and calculate the proportion of cases that include such events.

5. **Customer Satisfaction**:
   - **Measurable**: This KPI may require additional data (e.g., survey results) that are not present in the event logs.

Next, we will implement the calculations for these KPIs based on the available data and validate their SMART criteria. Let’s start with the Cycle Time KPI. We'll calculate the average cycle time for both Domestic and International declarations.

Let's proceed with the calculation for the Cycle Time KPI.

It looks like there was an error because the `get_case_duration_avg` attribute does not exist in the `pm4py.statistics.traces.generic.log.case_statistics` module. 

We can calculate the cycle time differently using the timestamps in the event logs. Specifically, we can calculate the duration from the start to the end of each case. Let's use the first and last events of each case to compute these durations.

Let's proceed with calculating the average cycle time using the correct approach.

It seems there's an issue with the module import. Let's try a different approach using available functions for calculating case durations directly from the log.

We'll calculate the start and end times for each case and then determine the duration. Let's proceed with this approach.

Here are the calculated average cycle times:

- **Domestic Declarations**: Approximately 276.61 hours
- **International Declarations**: Approximately 2074.92 hours

These values indicate the average time taken to complete a declaration from start to finish.

Next steps:
1. **Conformance Rate**:
   - We'll use process mining techniques to compare the actual event log to a reference model.
2. **Throughput Rate**:
   - We'll count the number of declarations processed within specific time periods.
3. **Rework Rate**:
   - We'll identify and calculate the proportion of cases that include rework events.

Would you like to proceed with calculating the Conformance Rate next, or focus on one of the other KPIs?

GraphRecursionError: Recursion limit of 10 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.